# Creating Initial Balance lines

The initial balance is the first hour of the regular trading session. In U.S. Eastern Time this is 09:30–10:30. Many traders consider this first hour special because it often sets the day’s early high and low and defines the early character of the day. We use the initial balance to locate strong bursts that may shape intraday direction.

So, we need to determine the highest and lowest price levels of the first hour in each U.S. Stock Market trading day.

- We have 55 trading days which means there must be 55 Initial Balance pairs with high/low levels.

In [1]:
from pathlib import Path
import pandas as pd